In [ ]:
# Importación de librerias.
import pandas as pd
from gensim.models import KeyedVectors
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
hiperonimos_df = pd.read_csv('hiperonimos_animales.csv')
hiperonimos_df.head()

In [ ]:
embeddings_WIKI = KeyedVectors.load_word2vec_format('embeddings/wiki.es.vec', limit=None)
embeddings_SUC = KeyedVectors.load_word2vec_format('embeddings/embeddings-l-model.vec', limit=None)
embeddings_SWOW = KeyedVectors.load_word2vec_format('embeddings/swow.embedding.was.26-04-2022.vec', limit=None)
lower_embeddings_SWOW = KeyedVectors(vector_size=embeddings_SWOW.vector_size)
embeddings_SBWC = KeyedVectors.load_word2vec_format('embeddings/SBW-vectors-300-min5.txt', limit=None)
embeddings_Glove = KeyedVectors.load_word2vec_format('embeddings/glove-sbwc.i25.vec', limit=None)

In [ ]:
# Embedding SWOW pasado a minusculas 

# Vectores y palabras del SWOW.
palabras = list(embeddings_SWOW.key_to_index.keys())
vectores = list(embeddings_SWOW.vectors)

# Convertir todas las palabras a minúsculas.
palabras_minusculas = [palabra.lower() for palabra in palabras]

# Creación de un nuevo modelo SWOW con las palabras y vectores en minúsculas. 
lower_embeddings_SWOW = KeyedVectors(vector_size=embeddings_SWOW.vector_size)
lower_embeddings_SWOW.add_vectors(palabras_minusculas, vectores)

In [ ]:
lista_de_Embeddings = [embeddings_WIKI, embeddings_SUC, lower_embeddings_SWOW, embeddings_SBWC, embeddings_Glove]

In [ ]:
# Filtrado de palabras en todos los embeddings
hiperonimos_df = hiperonimos_df.dropna()
filtro_palabra = [all(word in e.key_to_index for e in lista_de_Embeddings) for word in hiperonimos_df['palabra']]
filtro_hiperonimo = [all(word in e.key_to_index for e in lista_de_Embeddings) for word in hiperonimos_df['hiperonimo']]

# Combinando ambos filtros
filtro = [a and b for a, b in zip(filtro_palabra, filtro_hiperonimo)]

hiperonimos_df = hiperonimos_df[filtro]

In [ ]:
palabras_no_encontradas2 = []

def calcular_predicciones_hiperonimos(embedding, hiperonimos_df, K_fijo):
    predicciones = []
    for index, row in hiperonimos_df.iterrows():
        palabra = row['palabra']

        try:
            resultado = embedding.most_similar(positive=[palabra], topn=K_fijo)
            predicciones.append([palabra for palabra, _ in resultado])
        except KeyError as e:
            print(f"Una de las palabras no está en el embedding: {e}")
            predicciones.append([])
            if palabra not in palabras_no_encontradas2:
                palabras_no_encontradas2.append(palabra)
    return predicciones


In [ ]:
def metricas_topK_hiperonimos(predicciones, hiperonimos_df, K):
    topK_metricas = []
    for i, prediccion in enumerate(predicciones):
        hiperonimo_real = hiperonimos_df.iloc[i]['hiperonimo'].lower()
        topK_metricas.append(int(hiperonimo_real in prediccion[0:K]))
    return np.mean(topK_metricas)


In [ ]:
todas_predicciones_hiperonimos = [calcular_predicciones_hiperonimos(e, hiperonimos_df, K_fijo) for e in lista_de_Embeddings]

topK_metricas_hiperonimos = [[metricas_topK_hiperonimos(prediccion, hiperonimos_df, K) for K in Ks] for prediccion in todas_predicciones_hiperonimos]

print(f"Métricas top-K ={Ks}: {topK_metricas_hiperonimos}")